In [ ]:
# Example notebook demonstrating how to use the phenomic-ai package to infer embeddings wrt. a single cell RNA .h5ad file

In [ ]:
! pip install phenomic-ai  # install/update phenomic-ai dependency/package  # https://pypi.org/project/phenomic-ai/

In [6]:
import pai  # import phenomic-ai dependency/package using pai alias

In [9]:
tmp_dir = "/home/ubuntu/tmp/pai/embed"  # temporary directory is the root output directory where the downloaded zip files (zips/) and unzipped directories (results/) will be output

In [16]:
print(pai.utils.option_choices.tissue_organ_option_choices)  # tissue/organ option choices

['adipose', 'adrenal gland', 'bile duct', 'bladder', 'blood', 'brain', 'breast', 'colorectal', 'eye', 'head and neck', 'heart', 'intestine', 'kidney', 'liver', 'lung', 'lymph-node', 'marrow', 'muscle', 'ovaries', 'pancreas', 'placenta', 'pleura', 'prostate', 'salivary gland', 'skin', 'spleen', 'stomach', 'testis', 'thymus', 'thyroid gland', 'uterus', 'vasculature']


In [13]:
pai_embeddings = pai.embed.PaiEmbeddings(tmp_dir=tmp_dir)  # init w/ temporary directory

In [11]:
h5ad_path = "/home/ubuntu/tmp/pai/embed/adatas/anndata.h5ad"  # h5ad path is the path to the single cell RNA .h5ad file intended to be uploaded and embeded
tissue_organ = "adipose"  # tissue/organ specifies the tissue/organ associated wrt. the single cells

In [12]:
pai_embeddings.inference(h5ad_path=h5ad_path, tissue_organ=tissue_organ)

09/11/2024 03:30:52 PM Checking destination folders...
09/11/2024 03:30:52 PM Uploading h5ad file...
09/11/2024 03:30:52 PM Getting upload id
09/11/2024 03:30:52 PM Recieved uuid: 104c821f-6c64-49ab-b496-630341329be8
09/11/2024 03:30:53 PM Upload complete, job id: febf1176add5420b8e34c0c3c536bdf1
09/11/2024 03:30:53 PM Listening for job status
09/11/2024 03:30:53 PM Job status: SUBMITTED
09/11/2024 03:30:58 PM Job status: VALIDATING
09/11/2024 03:31:03 PM Job status: RUNNING
09/11/2024 03:31:08 PM Job status: COMPLETED
09/11/2024 03:31:08 PM Downloading job
